# 自动微分
- 自动微分(automatic differentiation)：深度学习框架中自动计算导数的方法
- 计算图(computational graph)：跟踪计算是哪些数据通过哪些组合操作产生输出
- 反向传播(backpropagate)：跟踪整个computational graph，填充关于每个参数的偏导数
  
  实际中，根据设计好的模型，系统会构建一个computational graph，并且自动微分使系统能够随后反向传播梯度

## 例子
对于函数$y=2\mathbf{X}^\top \mathbf{X}$关于列向量$\mathbf{X}$求导

In [10]:
# 创建X并为其分配初始值
import torch
x=torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [11]:
# 需要一个地方存储梯度，并且要在原地更改
# 等价于x=torch.arange(4.0,requires_grad=True)  
x.requires_grad_(True) 
x.grad

In [12]:
# 计算y
y=2*torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

In [13]:
# 计算并打印梯度
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [14]:
x.grad==4*x

tensor([True, True, True, True])

In [15]:
# 默认pytroch会积累梯度，需要清零
x.grad.zero_()
y=x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## 非scalar变量的backpropogation
当y不是scalar时，向量y关于向量x的导数是一个矩阵。对于高阶和高维的y和x，求导的结果可以是一个高阶tensor。但当调用向量的反向计算时，我们通常会试图计算一批训练样本中每个组成部分的损失函数的导数。 这里，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。

In [16]:
# 非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度
# 本例只想求偏导数的和，所以传递梯度1
x.grad.zero_()
y=x*x
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])